In [1]:
import numpy
import cv2
import os
from tqdm import tqdm

In [2]:
path = "../input/dynamic-sign-language-gestures-01/all"
image_size = 256
frame_rate = 0.1

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (image_size, image_size))
    return [hasFrames, image]

In [3]:
videos_list = sorted(os.listdir(path))[:500]
new_list = []

for xyz in tqdm(range(10)):
    mno = xyz*50
    for index in range(mno, mno+50):
        new_list.append(videos_list[index])

100%|██████████| 10/10 [00:00<00:00, 48044.72it/s]


In [4]:
frames = []
labels = []

for video in tqdm(new_list):
#     vidcap = cv2.VideoCapture(path + '/' + video)
#     sec = 0
#     mini = []
#     success, frame = getFrame(sec)
#     while success:
#         mini.append(frame)
#         sec = sec + frame_rate
#         sec = round(sec, 2)
#         success, frame = getFrame(sec)
        
#     frames.append(mini[-8:])
    labels.extend([int(video[:3]) -1]*8)

100%|██████████| 500/500 [00:00<00:00, 315788.59it/s]


In [5]:
from tensorflow.keras.utils import to_categorical
# frames = numpy.array(frames)
frames = numpy.load('../input/aslframes/last8-10classes.npy')
labels = numpy.array(labels)
cat_labels = to_categorical(labels)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(frames, cat_labels, test_size=0.1,
                                                   shuffle=True, random_state=42, stratify=labels)

In [7]:
import tensorflow
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base_model = ResNet50(include_top=False)
embed = GlobalMaxPooling2D()(base_model.output)
layer = Dropout(0.4)(embed)
layer = Dense(1024, activation='relu')(layer)
layer = Dropout(0.4)(layer)
layer = Dense(256, activation='relu')(layer)
layer = Dense(64, activation='relu')(layer)
output = Dense(10, activation='softmax')(layer)

model = Model(inputs=base_model.inputs, outputs=output)

83689472/83683744 [==============================] - 1s 0us/step


In [8]:
base_model.trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(filepath='model.h5', monitor='val_accuracy',
                                                       save_best_only=True, verbose=1)
earlyStop = tensorflow.keras.callbacks.EarlyStopping(patience=16, monitor='val_accuracy',
                                                    restore_best_weights=True, verbose=1)
reduceLR = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, 
                                                patience=4, verbose=1)

model.fit(x_train, y_train, batch_size=32, epochs=128, verbose=1,
          validation_data = (x_test, y_test),
          callbacks=[checkpoint, earlyStop, reduceLR])

Epoch 1/128
113/113 [==============================] - 10s 42ms/step - loss: 14.7050 - accuracy: 0.1045 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 00001: val_accuracy improved from -inf to 0.10000, saving model to model.h5
Epoch 2/128
113/113 [==============================] - 4s 32ms/step - loss: 2.3055 - accuracy: 0.1029 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 00002: val_accuracy did not improve from 0.10000
Epoch 3/128
113/113 [==============================] - 4s 31ms/step - loss: 2.3032 - accuracy: 0.1049 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 00003: val_accuracy did not improve from 0.10000
Epoch 4/128
113/113 [==============================] - 4s 31ms/step - loss: 2.3028 - accuracy: 0.1017 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 00004: val_accuracy did not improve from 0.10000
Epoch 5/128
113/113 [==============================] - 4s 31ms/step - loss: 2.3033 - accuracy: 0.1042 - val_loss: 2.3026 - val_accuracy: 0.1000

Epoch 00005: val_accuracy 

In [9]:
base_model.trainable = True
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(filepath='model.h5', monitor='val_accuracy',
                                                       save_best_only=True, verbose=1)
earlyStop = tensorflow.keras.callbacks.EarlyStopping(patience=16, monitor='val_accuracy',
                                                    restore_best_weights=True, verbose=1)
reduceLR = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, 
                                                patience=4, verbose=1)

model.fit(x_train, y_train, batch_size=32, epochs=128, verbose=1,
          validation_data = (x_test, y_test),
          callbacks=[checkpoint, earlyStop, reduceLR])

Epoch 1/128
113/113 [==============================] - 20s 139ms/step - loss: 2.0381 - accuracy: 0.2034 - val_loss: 78.5682 - val_accuracy: 0.1075

Epoch 00001: val_accuracy improved from -inf to 0.10750, saving model to model.h5
Epoch 2/128
113/113 [==============================] - 15s 130ms/step - loss: 0.8116 - accuracy: 0.6905 - val_loss: 2.1605 - val_accuracy: 0.7200

Epoch 00002: val_accuracy improved from 0.10750 to 0.72000, saving model to model.h5
Epoch 3/128
113/113 [==============================] - 15s 130ms/step - loss: 0.4591 - accuracy: 0.8456 - val_loss: 2.5302 - val_accuracy: 0.6750

Epoch 00003: val_accuracy did not improve from 0.72000
Epoch 4/128
113/113 [==============================] - 15s 130ms/step - loss: 0.4843 - accuracy: 0.8786 - val_loss: 2.3436 - val_accuracy: 0.6850

Epoch 00004: val_accuracy did not improve from 0.72000
Epoch 5/128
113/113 [==============================] - 15s 130ms/step - loss: 0.2461 - accuracy: 0.9396 - val_loss: 0.3819 - val_accur

In [10]:
y_pred = model.predict(x_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1))

1.0

In [11]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

array([[40,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 40,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 40,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 40,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 40,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 40,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 40,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 40,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 40,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 40]])